In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
celeba_bldr = tfds.builder('celeb_a')
celeba_bldr.download_and_prepare()
celeba_ds = celeba_bldr.as_dataset(shuffle_files=False)
celeba_train = celeba_ds['train']
celeba_val = celeba_ds['validation']
celeba_test = celeba_ds['test']

In [3]:
celeba_train = celeba_train.take(16000)
celeba_val = celeba_val.take(1000)

In [4]:
def preprocess_augmentation(example, size=(64, 64), mode='train'):
    image = example['image']
    label = example['attributes']['Male']

    if mode == 'train':
        image_cropped = tf.image.random_crop(image, size=(178, 178, 3))
        image_resized = tf.image.resize(image_cropped, size=size)
        image_flipped = tf.image.random_flip_left_right(image_resized)
        return image_flipped/255.0, tf.cast(label, tf.int32) 
    else:
        image_cropped = tf.image.crop_to_bounding_box(image, offset_height=20, offset_width=0, target_height=178, target_width=178)
        image_resized = tf.image.resize(image_cropped,size=size)
        return image_resized/255.0, tf.cast(label, tf.int32)

In [5]:
next(iter(celeba_train))['image']

<tf.Tensor: shape=(218, 178, 3), dtype=uint8, numpy=
array([[[ 30,  32,  21],
        [ 30,  32,  21],
        [ 31,  33,  22],
        ...,
        [ 19,  19,  17],
        [ 10,  10,   8],
        [ 10,  11,   6]],

       [[ 17,  19,   8],
        [ 18,  20,   9],
        [ 19,  21,  10],
        ...,
        [ 22,  22,  20],
        [ 15,  15,  13],
        [ 15,  16,  11]],

       [[  5,   6,   0],
        [  6,   7,   0],
        [  7,   8,   0],
        ...,
        [ 27,  27,  25],
        [ 22,  22,  20],
        [ 22,  23,  18]],

       ...,

       [[107,  92,  89],
        [ 73,  58,  55],
        [ 55,  37,  33],
        ...,
        [ 40,  36,  25],
        [ 63,  59,  48],
        [ 65,  61,  50]],

       [[106,  91,  88],
        [ 72,  57,  54],
        [ 50,  32,  28],
        ...,
        [ 54,  48,  36],
        [ 73,  69,  58],
        [ 73,  69,  58]],

       [[113,  98,  95],
        [ 78,  63,  60],
        [ 55,  37,  33],
        ...,
        [ 54,  48,  3

In [6]:
tf.random.set_seed(42)
IMAGE_SIZE=(150,150)
celeba_train = celeba_train.map(lambda x: preprocess_augmentation(x, size=IMAGE_SIZE, mode='train'))
celeba_train = celeba_train.shuffle(buffer_size=2000).repeat().batch(32)

celeba_val = celeba_val.map(lambda x: preprocess_augmentation(x, size=IMAGE_SIZE, mode='eval'))
celeba_val = celeba_val.batch(32)


In [7]:
next(iter(celeba_train))[0]

<tf.Tensor: shape=(32, 150, 150, 3), dtype=float32, numpy=
array([[[[0.37328106, 0.24779086, 0.3928889 ],
         [0.3814902 , 0.25599998, 0.40109804],
         [0.38248366, 0.25699347, 0.4020915 ],
         ...,
         [0.39607844, 0.2784314 , 0.41960785],
         [0.3989019 , 0.28125486, 0.42807826],
         [0.4       , 0.28235295, 0.43137255]],

        [[0.37328106, 0.24779086, 0.3928889 ],
         [0.3814902 , 0.25599998, 0.40109804],
         [0.38248366, 0.25699347, 0.4020915 ],
         ...,
         [0.39607844, 0.2784314 , 0.41960785],
         [0.3989019 , 0.28125486, 0.42807826],
         [0.4       , 0.28235295, 0.43137255]],

        [[0.37328106, 0.24779086, 0.3928889 ],
         [0.3814902 , 0.25599998, 0.40109804],
         [0.38248366, 0.25699347, 0.4020915 ],
         ...,
         [0.39607844, 0.2784314 , 0.41960785],
         [0.3989019 , 0.28125486, 0.42807826],
         [0.4       , 0.28235295, 0.43137255]],

        ...,

        [[0.6784314 , 0.6745098 ,

In [8]:
conv_base = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
network = tf.keras.models.Sequential([
    conv_base,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
network.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2097408   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


In [9]:
conv_base.trainable = False

In [10]:
network.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.RMSprop(learning_rate=2e-5), metrics=['accuracy'])
history = network.fit(celeba_train, epochs=10, validation_data=celeba_val, steps_per_epoch=np.ceil(16000/32))
history.history

Epoch 1/10
500/500 [==============================] - 553s 1s/step - loss: 0.4222 - accuracy: 0.8095 - val_loss: 0.2402 - val_accuracy: 0.9070
Epoch 2/10
  8/500 [..............................] - ETA: 8:50 - loss: 0.3095 - accuracy: 0.8902

KeyboardInterrupt: 